# first try

In [48]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
import pandas as pd
import xgboost
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier


X = pd.read_csv('data/X.csv')
y = pd.read_csv('data/y.csv')

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# used for kaggle submission
X_kaggle = pd.read_csv('data/X_kaggle.csv')

### XGBoost ###

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score
import xgboost

# Load the data
X = pd.read_csv('data/X.csv')
y = pd.read_csv('data/y.csv')

# Train, validation, and test split (60% train, 20% validation, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% train
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test

# Used for Kaggle submission
X_kaggle = pd.read_csv('data/X_kaggle.csv')

# Define the parameter grid
kf = KFold(n_splits=5)

param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'random_state': [42],
    'subsample': [0.8, 1]
}

# Initialize the XGBClassifier
xgb_model = xgboost.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=kf, scoring='accuracy', verbose=1)

# Fit GridSearchCV with training data
grid_search.fit(X_train, y_train)

# Best XGB model after GridSearch
best_xgb_model = grid_search.best_estimator_
print(f"Best GridSearchCV Score on Training Data: {grid_search.best_score_}")

# Evaluate on the validation set to check for overfitting
y_pred_val = best_xgb_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print(f"Accuracy on Validation Set: {val_accuracy}")

# Predict on the test set
y_pred_test = best_xgb_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy on Test Set: {test_accuracy}")

# Predict on the Kaggle dataset for submission
y_pred_xgb = best_xgb_model.predict(X_kaggle)

# Prepare submission file
y_pred_xgb = pd.DataFrame(y_pred_xgb, columns=['prediction'])
y_pred_xgb['ID'] = y_pred_xgb.index
y_pred_xgb

# Optional: Save predictions to CSV for submission
y_pred_xgb.to_csv('y_pred_xgb.csv', index=False)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best GridSearchCV Score on Training Data: 0.9271885521885522
Accuracy on Validation Set: 0.9345959595959596
Accuracy on Test Set: 0.927020202020202


In [26]:
print(y_pred_xgb['prediction'].value_counts())

prediction
1    775
0    691
Name: count, dtype: int64


In [67]:
# Define the parameter grid
kf = KFold(n_splits=5)

param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'random_state': [42],
    'subsample': [0.8, 1]
}

# Initialize the XGBClassifier
xgb_model = xgboost.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=kf, scoring='accuracy', verbose=1)

# # Fit GridSearchCV
# grid_search.fit(X_train, y_train)
# best_xgb_model = grid_search.best_estimator_
# print(grid_search.best_score_)

# # Predict on the test set
# y_pred_test = best_xgb_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred_test)
# print(f"Accuracy on test set: {accuracy}")

y_pred_xgb = xgb_model.predict(X_kaggle)

y_pred_xgb = pd.DataFrame(y_pred_xgb, columns=['prediction'])
y_pred_xgb['ID'] = y_pred_xgb.index
y_pred_xgb

,prediction,ID
0,0,0
1,1,1
2,1,2
3,1,3
4,0,4
...,...,...
1461,1,1461
1462,1,1462
1463,0,1463
1464,1,1464


In [68]:
print(y_pred_xgb['prediction'].value_counts())

prediction
1    791
0    675
Name: count, dtype: int64


In [69]:
y_pred_xgb.to_csv('y_pred_xgb2.csv', index=False)

### LazyClassifier ###

We can check which top 10 models are the best performing and furtherly train them ourselves with more delicate steps, meaning GridSearch and kFold :)

In [31]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=accuracy_score)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|███████████████████████████████████████████| 29/29 [00:44<00:00,  1.52s/it]

[LightGBM] [Info] Number of positive: 5274, number of negative: 10566
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000618 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1456
[LightGBM] [Info] Number of data points in the train set: 15840, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.332955 -> initscore=-0.694852
[LightGBM] [Info] Start training from score -0.694852
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LGBMClassifier                     0.94               0.92     0.92      0.94   
XGBClassifier                      0.94               0.92     0.92      0.94   
RandomForestClassifier             0.93               0.92     0.92      0.93   
ExtraTreesClassifier               0.9

### Random Forest ###

In [62]:
# Define the parameter grid
kf = KFold(n_splits=5)

param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['sqrt', 'log2']
}


# Initialize the Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=3, scoring='accuracy', verbose=1)

# Fit GridSearchCV
grid_search_rf.fit(X_train, y_train)

best_rf_model = grid_search_rf.best_estimator_
print(grid_search_rf.best_score_)

y_pred_rf = best_rf_model.predict(X_kaggle)

y_pred_rf = pd.DataFrame(y_pred_rf, columns=['prediction'])
y_pred_rf['ID'] = y_pred_rf.index
y_pred_rf

Fitting 3 folds for each of 72 candidates, totalling 216 fits
0.9110479797979799


,prediction,ID
0,0,0
1,1,1
2,1,2
3,1,3
4,0,4
...,...,...
1461,1,1461
1462,1,1462
1463,0,1463
1464,1,1464


In [59]:
y_pred_rf['prediction'].value_counts()

prediction
1    871
0    595
Name: count, dtype: int64

In [60]:
y_pred_rf.to_csv('y_pred_rf2.csv', index=False)

### ExtraTreesClassifier ###

In [37]:
param_grid_etc = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['sqrt', 'log2']
}


# Initialize the ExtraTreeClassifier
etc_model = ExtraTreesClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_etc = GridSearchCV(estimator=etc_model, param_grid=param_grid_etc, cv=kf, scoring='accuracy', verbose=1)

# Fit GridSearchCV
grid_search_etc.fit(X_train, y_train)

best_etc_model = grid_search_etc.best_estimator_
print(grid_search_etc.best_score_)

y_pred_etc = best_etc_model.predict(X_kaggle)

y_pred_etc = pd.DataFrame(y_pred_etc, columns=['prediction'])
y_pred_etc['ID'] = y_pred_etc.index
y_pred_etc

Fitting 5 folds for each of 108 candidates, totalling 540 fits
0.8874053030303031


,prediction,ID
0,0,0
1,1,1
2,1,2
3,0,3
4,1,4
...,...,...
1461,1,1461
1462,1,1462
1463,0,1463
1464,1,1464


### LGBMClassifier ###

In [37]:
param_grid_lgb = {
    'num_leaves': [25, 30, 50],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}


# Initialize the LGBMClassifier
lgb_model = lgb.LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_lgb = GridSearchCV(estimator=lgb_model, param_grid=param_grid_lgb, cv=3, scoring='accuracy', verbose=1)

# Fit GridSearchCV
grid_search_lgb.fit(X_train, y_train)

best_lgb_model = grid_search_lgb.best_estimator_
print(grid_search_lgb.best_score_)

y_pred_lgb = best_lgb_model.predict(X_kaggle)

y_pred_lgb = pd.DataFrame(y_pred_lgb, columns=['prediction'])
y_pred_lgb['ID'] = y_pred_lgb.index
y_pred_lgb

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[LightGBM] [Info] Number of positive: 3516, number of negative: 7044
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000523 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1462
[LightGBM] [Info] Number of data points in the train set: 10560, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.332955 -> initscore=-0.694852
[LightGBM] [Info] Start training from score -0.694852
[LightGBM] [Info] Number of positive: 3516, number of negative: 7044
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000407 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1508
[LightGBM] [Info] Number of data points in the train

,prediction,ID
0,0,0
1,1,1
2,1,2
3,1,3
4,0,4
...,...,...
1461,1,1461
1462,1,1462
1463,0,1463
1464,1,1464


In [39]:
y_pred_lgb['prediction'].value_counts()

prediction
1    785
0    681
Name: count, dtype: int64

In [33]:
y_pred_lgb.to_csv('data/y_pred_lgb.csv', index=False)

### AdaBoostClassifier ###

In [41]:
param_grid_ada = {
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'n_estimators': [10, 50, 100, 500],
}


# Initialize the AdaBoostClassifier
ada_model = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_ada = GridSearchCV(estimator=ada_model, param_grid=param_grid_ada, cv=kf, scoring='accuracy', verbose=1)

# Fit GridSearchCV
grid_search_ada.fit(X_train, y_train)

best_ada_model = grid_search_ada.best_estimator_
print(grid_search_ada.best_score_)

y_pred_ada = best_ada_model.predict(X_kaggle)

y_pred_ada = pd.DataFrame(y_pred_ada, columns=['prediction'])
y_pred_ada['ID'] = y_pred_ada.index
y_pred_ada

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0.8826704545454545


,prediction,ID
0,0,0
1,1,1
2,0,2
3,0,3
4,1,4
...,...,...
1461,0,1461
1462,1,1462
1463,0,1463
1464,1,1464


In [ ]:
print("slay the boots down houston i'm deceased")

### Final files to submit (per model) ###

In [42]:
# XGBoost
y_pred_xgb.to_csv('data/y_pred_xgb.csv', index=False)

# Ramdom Forest
y_pred_rf.to_csv('data/y_pred_rf.csv', index=False)

# LightGBM
y_pred_lgb.to_csv('data/y_pred_lgb.csv', index=False)